# Twitter with twarc
A UCSB origihnal Carpentry workshop

In [1]:
# this version generally has calls to the twitter api commented
# out to preserve quotas

In [2]:
# administravia
# upon re-start we need to install twarc2 extensions
! pip install twarc-csv
! pip install emoji

# Episode 2
You should have a taxday.jsonl file

In [3]:
# BASH commands start with a BANG!
!twarc2 --help

Usage: twarc2 [OPTIONS] COMMAND [ARGS]...

  Collect data from the Twitter V2 API.

Options:
  --consumer-key TEXT         Twitter app consumer key (aka "App Key")
  --consumer-secret TEXT      Twitter app consumer secret (aka "App Secret")
  --access-token TEXT         Twitter app access token for user
                              authentication.
  --access-token-secret TEXT  Twitter app access token secret for user
                              authentication.
  --bearer-token TEXT         Twitter app access bearer token.
  --app-auth / --user-auth    Use application authentication or user
                              authentication. Some rate limits are higher with
                              user authentication, but not all endpoints are
                              supported.  [default: app-auth]
  -l, --log TEXT
  --verbose
  --metadata / --no-metadata  Include/don't include metadata about when and
                              how data was collected.  [default: metadata]
  

In [4]:
#  what libraries will we need to be loading in our notebook?
#  we need to always distinguish between 
#  running BASH vs. running a line of python.

import pandas
import twarc_csv
import textblob
import nltk
import os
import emoji

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# this comes into play for ep 8
!python -m textblob.download_corpora
nltk.download('stopwords')

[nltk_data] Downloading package brown to /home/jovyan/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /home/jovyan/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
# and of course, it's important to know where we are working
# I can send a BASH command from my notebook with a !:
!pwd

/home/jovyan/twarc_run


In [ ]:
# you can also do this with Python
os.getcwd()

In [ ]:
# make sure we are in the twarc_run folder
os.chdir("twarc_run")

In [ ]:
os.getcwd()

## Running twarc
Let's get the timeline of one of twarc's creators.

In [ ]:
# !twarc2 timeline BergisJules > raw_data/bjules.jsonl

In [7]:
# timeline objects need to be flattened in order to be analyzed as tweets
!twarc2 flatten raw_data/bjules.jsonl output_data/bjules_flat.jsonl

100%|██████████████| Processed 8.95M/8.95M of input file [00:00<00:00, 10.1MB/s]


### Challenge 1
- Can you find the file called “bjules_flat.jsonl”?
- What's the timestamp on the first one. The last one?

- How many tweets did you get from Bergis? (we can't tell without flattening)
- Download a timeline for a person of your choice. How many tweets did you get? 
- What’s the oldest one?

In [8]:
!twarc2 flatten raw_data/ecodatasci.jsonl > output_data/ecodatasci_flat.jsonl
!wc output_data/ecodatasci_flat.jsonl

    473  204893 2875654 output_data/ecodatasci_flat.jsonl


## Convert to csv

In [9]:
!twarc2 csv raw_data/bjules.jsonl output_data/bjules.csv

100%|██████████████| Processed 8.95M/8.95M of input file [00:02<00:00, 3.62MB/s]

ℹ️
Parsed 3140 tweets objects from 33 lines in the input file.
Wrote 3140 rows and output 74 columns in the CSV.



### Challenge 2

In [12]:
# commented line is a solution to challenge 1
# !twarc2 timeline ecodatasci > raw_data/ecodatasci.jsonl

!twarc2 flatten raw_data/ecodatasci.jsonl > output_data/ecodatasci_flat.jsonl
!twarc2 csv output_data/ecodatasci_flat.jsonl > output_data/ecodatasci.csv 
ecodatasci_df = pandas.read_csv("output_data/ecodatasci.csv")


# Episode 3: examining tweets
What comes along with a tweet
- Look at one_tweet in Jupyter viewer
- Look at one_tweet with nano
- Look at tweet as csv
- Look at all the entities of a tweet

## To flatten or not flatten

In [17]:
### Let's look at a tweet as a csv:

#this doesn't work
!twarc2 csv raw_data/one_tweet.jsonl output_data/one_tweet.csv

# it needs to be flattened before csv can be made?
# but this also doesn't work:
!twarc2 flatten raw_data/one_tweet.jsonl output_data/one_tweet_flat.jsonl



100%|█████████████████| Processed 299k/299k of input file [00:00<00:00, 134MB/s]

ℹ️
Parsed 0 tweets objects from 35 lines in the input file.
34 failed to parse. See twarc.log for details.
Wrote 0 rows and output 74 columns in the CSV.

  0%|                        | Processed 0.00/299k of input file [00:00<?, ?B/s]
⚡ Expecting property name enclosed in double quotes: line 2 column 1 (char 39)


### Make your jsonl 1 tweet per line
Flattening will let you do our most basic unix-y analysis, turn
timelines into countable lists, and enable you to run twarc1
utilities later on in the workshop

In [19]:
!wc raw_data/taxday.jsonl

    15698   7653538 100048736 raw_data/taxday.jsonl


A straight harvest using search or stream doesn't need to be flattened 
to do our most basic analysis: wc.

In [18]:
# flatten
!twarc2 flatten raw_data/taxday.jsonl output_data/taxday_flat.jsonl

100%|██████████████| Processed 95.4M/95.4M of input file [00:03<00:00, 25.1MB/s]


In [20]:
!wc output_data/taxday_flat.jsonl

    15698   7653538 100048736 output_data/taxday_flat.jsonl


For taxday, it doesn't seem to matter if we flatten or not

## When we look at bjules, we really do need to flatten it.

In [21]:
! wc raw_data/bjules.jsonl

     33  844925 9385982 raw_data/bjules.jsonl


33 lines doesn't mean 33 tweets. I suspected there was more there because
I got an error message about hitting a limit of 3200. 

And below, the csv converter tells us there are 3140 tweets.

In [22]:
# convert
!twarc2 csv raw_data/bjules.jsonl output_data/bjules.csv

100%|██████████████| Processed 8.95M/8.95M of input file [00:02<00:00, 3.74MB/s]

ℹ️
Parsed 3140 tweets objects from 33 lines in the input file.
Wrote 3140 rows and output 74 columns in the CSV.



In [24]:
# When I did this, I got 3166 tweets (as opposed to the 33 lines that the original file was)
! wc output_data/bjules_flat.jsonl
# ! wc output_data/bjules.csv

    3140  1717206 23260288 output_data/bjules_flat.jsonl


The csv is 1 line longer because it has column headers.

## Next harvest
Next we'll get just Bergis' original content. 
In other words, only the tweets that he wrote, not
any retweets or replied to other people tweets.

Can we go back further on his timeline by looking
only for Bergis's original content?

Not really--it looks like the limit applies to the search,
not the results. 

But this does tell us that Jules is a prolific re-tweeter and/or replier. 

In [25]:
!twarc2 timeline BergisJules --exclude-retweets --exclude-replies > raw_data/bjules_original.jsonl

API limit of 3200 reached:   0%|             | 47/17680 [00:00<03:55, 74.85it/s]


In [26]:
!twarc2 flatten raw_data/bjules_original.jsonl output_data/bjules_original_flat.jsonl

100%|████████████████| Processed 107k/107k of input file [00:00<00:00, 25.2MB/s]


In [27]:
# save it as a csv so we can easily see the original writings of Jules
!twarc2 csv output_data/bjules_original_flat.jsonl output_data/bjules_original.csv

100%|████████████████| Processed 234k/234k of input file [00:00<00:00, 7.64MB/s]

ℹ️
Parsed 47 tweets objects from 47 lines in the input file.
Wrote 47 rows and output 74 columns in the CSV.



Let's look at out taxday.jsonl file that we prepared for you on April 18.

In [ ]:
!head -n 1 raw_data/taxday.jsonl

In [ ]:
# A lot comes along! 
!tail -n 1 raw_data/taxday.jsonl

## Challenge: tax day Tweets

In [28]:
# we harvested 3 hours worth of tweets for you on tax day.
# how many tweets?
!wc raw_data/taxday.jsonl
!wc output_data/taxday_flat.jsonl
# same number, so we don't have to flatten these.

    15698   7653538 100048736 raw_data/taxday.jsonl
    15698   7653538 100048736 output_data/taxday_flat.jsonl


In [ ]:
# maybe the last challenge for ep. 3 is examining this shorter file?

# Episode 4

In [29]:
# fishing around for good searches
# you can count without harvesting.
# kittens is an evergreen search. you should always see at lease
# dozens of mentions per hour
!twarc2 counts --text "kittens"

2022-05-13T22:23:05.000Z - 2022-05-13T23:00:00.000Z: 238
2022-05-13T23:00:00.000Z - 2022-05-14T00:00:00.000Z: 431
2022-05-14T00:00:00.000Z - 2022-05-14T01:00:00.000Z: 327
2022-05-14T01:00:00.000Z - 2022-05-14T02:00:00.000Z: 330
2022-05-14T02:00:00.000Z - 2022-05-14T03:00:00.000Z: 452
2022-05-14T03:00:00.000Z - 2022-05-14T04:00:00.000Z: 351
2022-05-14T04:00:00.000Z - 2022-05-14T05:00:00.000Z: 286
2022-05-14T05:00:00.000Z - 2022-05-14T06:00:00.000Z: 296
2022-05-14T06:00:00.000Z - 2022-05-14T07:00:00.000Z: 227
2022-05-14T07:00:00.000Z - 2022-05-14T08:00:00.000Z: 274
2022-05-14T08:00:00.000Z - 2022-05-14T09:00:00.000Z: 237
2022-05-14T09:00:00.000Z - 2022-05-14T10:00:00.000Z: 251
2022-05-14T10:00:00.000Z - 2022-05-14T11:00:00.000Z: 301
2022-05-14T11:00:00.000Z - 2022-05-14T12:00:00.000Z: 346
2022-05-14T12:00:00.000Z - 2022-05-14T13:00:00.000Z: 453
2022-05-14T13:00:00.000Z - 2022-05-14T14:00:00.000Z: 517
2022-05-14T14:00:00.000Z - 2022-05-14T15:00:00.000Z: 474
2022-05-14T15:00:00.000Z - 2022

In [ ]:
# recent search with granularity.
!twarc2 counts --granularity "day" --text "(#UCSBLibrary OR UCSBLibrary OR ucsblibrary OR #ucsblibrary OR davidsonlibrary OR #davidsonlibrary)"

In [ ]:
# phrase searching????
# this one isnt working
!twarc2 counts --granularity "day" --text "(#UCSB OR UCSB OR ucsb OR ("UC Santa Barbara"))"
!twarc2 counts --granularity "day" --text "("uc santa barbara")"
!twarc2 counts --granularity "day" --text "(#ucsb)"
!twarc2 counts --granularity "day" --text "(UCSB)"

In [ ]:
# search for hashtags when you really want hashtags. 
# search for a string returns both text and hashtage (an OR)
# NOT case sensitive
!twarc2 counts --granularity "day" --text "(#UCSB OR UCSB OR ucsb)"
!twarc2 counts --granularity "day" --text "(#ucsb)"
!twarc2 counts --granularity "day" --text "(UCSB)"

In [ ]:
## Endpoints: counts
!twarc2 counts --text "(Poker OR poker)" --granularity "day"
!twarc2 counts --text "(Golf OR golf)" --granularity "day"
!twarc2 counts --text "(Basketball OR basketball)" --granularity "day"
!twarc2 counts --text "(Baseball OR baseball)" --granularity "day"
!twarc2 counts --text "(Football OR football)" --granularity "day"

In [ ]:
## What's a lot?
!twarc2 counts --text "dog" --granularity "day"
!twarc2 counts --text "cat" --granularity "day"
!twarc2 counts --text "amazon" --granularity "day"
!twarc2 counts --text "right" --granularity "day"
!twarc2 counts --text "good" --granularity "day"


In [30]:
## a SFW timeline
# !twarc2 timeline ucsblibrary raw_data/library.jsonl
!twarc2 flatten raw_data/library.jsonl output_data/library.jsonl
!twarc2 csv output_data/library.jsonl output_data/library.csv
library_df = pandas.read_csv("output_data/library.csv")

100%|██████████████| Processed 5.31M/5.31M of input file [00:00<00:00, 10.9MB/s]
100%|██████████████| Processed 12.9M/12.9M of input file [00:02<00:00, 4.87MB/s]

ℹ️
Parsed 3226 tweets objects from 3226 lines in the input file.
Wrote 3226 rows and output 74 columns in the CSV.



In [ ]:
# confirm the dataframe's existance
# and view all column headers
list(library_df.columns)

### Converting to csv and dataframes

In [ ]:
# this line is not running
# flatten it first?
!twarc2 csv _data/taxday.jsonl output_data/taxday_flat.csv


## final challenge: Cats of Instagram
Let’s make a bigger datafile. Harvest 5000 tweets that use the hashtag “catsofinstagram” and put the dataset through the pipeline to answer the following questions:

- Did you get exactly 5000?
- How far back in time did you get?
- What is the most re-tweeted recent tweet on #catsofinstagram?
- Which person has the most number of followers in your dataset?
- Is it really a person?

In [33]:
# !twarc2 search --limit 5000 "#catsofinstagram" raw_data/hashtagcats.jsonl
!twarc2 csv raw_data/hashtagcats.jsonl > output_data/hashtagcats.csv
hashtagcats_df = pandas.read_csv("output_data/hashtagcats.csv")
! wc output_data/hashtagcats.csv
hashtagcats_df["created_at"].head()

    5090  1075715 19340796 output_data/hashtagcats.csv


0    2022-05-20T01:13:20.000Z
1    2022-05-20T01:13:08.000Z
2    2022-05-20T01:12:52.000Z
3    2022-05-20T01:12:49.000Z
4    2022-05-20T01:10:45.000Z
Name: created_at, dtype: object

In [ ]:
hashtagcats_df["created_at"].tail()

In [ ]:
hashtagcats_df

In [34]:
list(hashtagcats_df.columns)

['id',
 'conversation_id',
 'referenced_tweets.replied_to.id',
 'referenced_tweets.retweeted.id',
 'referenced_tweets.quoted.id',
 'author_id',
 'in_reply_to_user_id',
 'retweeted_user_id',
 'quoted_user_id',
 'created_at',
 'text',
 'lang',
 'source',
 'public_metrics.like_count',
 'public_metrics.quote_count',
 'public_metrics.reply_count',
 'public_metrics.retweet_count',
 'reply_settings',
 'possibly_sensitive',
 'withheld.scope',
 'withheld.copyright',
 'withheld.country_codes',
 'entities.annotations',
 'entities.cashtags',
 'entities.hashtags',
 'entities.mentions',
 'entities.urls',
 'context_annotations',
 'attachments.media',
 'attachments.media_keys',
 'attachments.poll.duration_minutes',
 'attachments.poll.end_datetime',
 'attachments.poll.id',
 'attachments.poll.options',
 'attachments.poll.voting_status',
 'attachments.poll_ids',
 'author.id',
 'author.created_at',
 'author.username',
 'author.name',
 'author.description',
 'author.entities.description.cashtags',
 'author

# Episode 5: Ethics & Twitter

In [36]:
# our first full-text analysis
# a list of words with TextBlob

# first we need to munge the data. remember from:
# list(library_df.columns)
# the tweet is library_df['text']

# TextBlob has its own data format.

# break tweets test column into a list, 
# then .join into one long string 
library_string = ' '.join(library_df['text'].tolist())
# turn the string into a blob
library_blob = textblob.TextBlob(library_string)


This produces a mess. 
Let's count the words and sort by their frequency of use:


In [37]:
library_freq = library_blob.word_counts
library_sorted_freq = sorted(library_freq.items(), 
                             key = lambda kv: kv[1], 
                             reverse = True)
library_sorted_freq[1:25]

[('https', 2282),
 ('to', 1646),
 ('of', 1523),
 ('and', 1244),
 ('ucsb', 1148),
 ('http', 1117),
 ('in', 1107),
 ('library', 1071),
 ('a', 1047),
 ('for', 1012),
 ('s', 745),
 ('on', 671),
 ('at', 634),
 ('you', 605),
 ('our', 547),
 ('we', 532),
 ('is', 499),
 ('from', 475),
 ('this', 453),
 ('ucsblibrary', 415),
 ('with', 391),
 ('by', 391),
 ('amp', 331),
 ('more', 327)]

We can at least get the english stopwords out. but this all didn't reall produce anything cleaner:

In [38]:
# load the stopwords to use
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')

In [39]:
# create a new text list that does
# NOT contain stopwords
library_str_stopped = [word for word in library_string.split() 
                       if word.lower() not in sw_nltk]
library_words_stopped = " ".join(library_str_stopped)

In [40]:
library_blob_stopped = textblob.TextBlob(library_words_stopped)
library_blob_stopped_freq = library_blob_stopped.word_counts
library_blob_stopped_sorted_freq = sorted(library_blob_stopped_freq.items(), 
                             key = lambda kv: kv[1], 
                             reverse = True)
library_blob_stopped_sorted_freq[1:50]

[('ucsb', 1148),
 ('http', 1117),
 ('library', 1071),
 ('s', 679),
 ('ucsblibrary', 415),
 ('amp', 331),
 ('new', 288),
 ('today', 250),
 ('us', 232),
 ('’', 228),
 ('book', 218),
 ('research', 212),
 ('here', 210),
 ('students', 195),
 ('reads', 189),
 ('open', 187),
 ('we', 178),
 ('check', 177),
 ('join', 164),
 ('week', 161),
 ('floor', 145),
 ('day', 143),
 ('free', 139),
 ('collections', 138),
 ('access', 136),
 ('art', 127),
 ('re', 126),
 ('special', 120),
 ('study', 119),
 ('learn', 116),
 ('collection', 112),
 ('first', 108),
 ('campus', 106),
 ('one', 103),
 ('info', 102),
 ('books', 102),
 ('available', 102),
 ('see', 100),
 ('uc', 99),
 ('come', 99),
 ('librarian', 98),
 ('community', 97),
 ('student', 97),
 ('talk', 96),
 ('exhibit', 95),
 ('hours', 93),
 ('more', 92),
 ('read', 90),
 ('science', 89)]

In [41]:
# a more meaningul segment
library_blob_stopped_sorted_freq[7:57]

[('new', 288),
 ('today', 250),
 ('us', 232),
 ('’', 228),
 ('book', 218),
 ('research', 212),
 ('here', 210),
 ('students', 195),
 ('reads', 189),
 ('open', 187),
 ('we', 178),
 ('check', 177),
 ('join', 164),
 ('week', 161),
 ('floor', 145),
 ('day', 143),
 ('free', 139),
 ('collections', 138),
 ('access', 136),
 ('art', 127),
 ('re', 126),
 ('special', 120),
 ('study', 119),
 ('learn', 116),
 ('collection', 112),
 ('first', 108),
 ('campus', 106),
 ('one', 103),
 ('info', 102),
 ('books', 102),
 ('available', 102),
 ('see', 100),
 ('uc', 99),
 ('come', 99),
 ('librarian', 98),
 ('community', 97),
 ('student', 97),
 ('talk', 96),
 ('exhibit', 95),
 ('hours', 93),
 ('more', 92),
 ('read', 90),
 ('science', 89),
 ('“', 87),
 ('”', 86),
 ('time', 86),
 ('barbara', 85),
 ('tbt', 84),
 ('santa', 83),
 ('year', 82)]

In [ ]:
type(library_stopped_blob)

In [ ]:
# Challenge: for the Python wizzes. #FIXME
# do that in a tidy way?
# what do pandas pipes look like?

In [42]:
# we made the utils folder during setup
# https://github.com/DocNow/twarc/tree/main/utils

!python utils/wall.py raw_data/hashtagcats.jsonl > output_data/hashtagcats.html

Traceback (most recent call last):
  File "/home/jovyan/twarc_run/utils/wall.py", line 147, in <module>
    url = tweet["user"]["profile_image_url"]
KeyError: 'user'


In [ ]:
# you gotta flatten that shit
!twarc2 flatten raw_data/hashtagcats.jsonl > output_data/hashtagcats_flat.jsonl
# !python utils/wall.py output_data/hashtagcats_flat.jsonl > output_data/hashtagcats.html

# or maybe not flatten.
# either way, I can't get it to run
! python utils/wall.py raw_data/hashtagcats.jsonl > output_data/cat_wall.html

## Challenge: Insta-rrectionists

In [ ]:
# this takes a very long time.
# !twarc2 hydrate raw_data/dehydratedCapitolRiotTweets.txt output_data/riots.jsonl

In [ ]:
# how long is this?
riots_dehydrated_df = pandas.read_csv("raw_data/dehydratedCapitolRiotTweets.txt"

# Episode 6: Working with our Tweets

1. retweets
1. emojis
1. wall
1. followers
1. retweeted-by

In [ ]:
# what does this do?
# it shows the tweet ID's of the Retweets in your dataset, and how much
!python utils/retweets.py raw_data/taxday.jsonl > output_data/taxday_retweets.csv

In [ ]:
# we gotta make sure that works with newly harvested stuff
!python utils/retweets.py raw_data/ucsblibrary_mentions.jsonl > output_data/ucsblibrary_mentions.csv

In [ ]:
!python utils/emojis.py raw_data/taxday.jsonl > output_data/taxday_emojis.csv

In [ ]:
# ! twarc2 mentions ucsblibrary raw_data/ucsblibrary_mentions.jsonl
! twarc2 csv raw_data/ucsblibrary_mentions.jsonl output_data/ucsblibrary_mentions.csv 
ucsb_library_mentions_df = pandas.read_csv('output_data/ucsblibrary_mentions.csv')

In [ ]:
# emojis for each of our datasets so far
# !python utils/emojis.py raw_data/hashtagcats.jsonl > output_data/hashtagcats_emojis.csv
!python utils/emojis.py raw_data/ucsblibrary_mentions.jsonl > output_data/ucsblibrary_mentions_emojis.csv

# Retweets vs. tweets
How much original content is there?
Do this for both taxday, library, and one of the cats.

In [ ]:
# with the util

In [ ]:
# via pandas and plottting
tax_retweets_df = pandas.read_csv("output_data/taxday_retweets.csv", names=["tweetid", "retweets"])
tax_retweets_df.head()
tax_retweets_df["retweets"].plot(kind = "hist")


In [ ]:
retweet_total = sum(tax_retweets_df["retweets"])


In [ ]:
tax_retweets_df["retweets"].plot(kind = "hist", loglog=True)

In [ ]:
# new beginning
# can I get a fresh filter and run emojis?
! twarc2 search "masked OR #masked" raw_data/masked.jsonl

In [ ]:
! twarc2 flatten raw_data/masked.jsonl output_data/masked_flat.jsonl 

In [ ]:
!python utils/emojis.py output_data/masked_flat.jsonl > output_data/masked_emojis.csv

# Episode 7: Search and Filter

In [ ]:
# use Twitter advanced search syntax (everthing in quotes!)
# to get tailored results
!twarc2 search --limit 800 "(cute OR fluffy OR haircut) (#catsofinstagram) lang:en" kittens.jsonl

In [ ]:
kittens_df = pandas.read_csv("output_data/kittens.csv")

In [ ]:
kittens_df

In [ ]:
list(kittens_df.columns)

# Episode 8: Python text analysis

In [ ]:
# what datafram is still here?
hashtagcats_df.columns

### Sentiment Analysis
To do this, we need to do a little Python

TextBlob is a text processing library that does sentiment analysis. 
The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

Before we use TextBlob for sentiment analysis, we need to download
datasets of words and their associated weights. These are called *corpora*.

In [ ]:
# commented out because I put it up in ep 2
# !python -m textblob.download_corpora

In [ ]:
# TextBlob needs a string, so this won't work.
textblob.TextBlob(hashtagcats_df).sentiment

In [ ]:
# even calling the column won't work:
textblob.TextBlob(hashtagcats_df['text']).sentiment

In [ ]:
# break tweets test column into a list, then .join into one long string 
hashtagcats_list = ' '.join(hashtagcats_df['text'].tolist())
# turn the string into a blob
hashtagcats_blob = textblob.TextBlob(hashtagcats_list)
# get the sentiment
hashtagcats_blob.sentiment

The overall sentiment of the language of kitty twitter is rather positive.
And the tweets tend to be subjective.

In [ ]:
# What do you think the sentiment of tax day might be?
# get the overall sentiment and see if it matches your prediction.

# Episode 9: Data Management

# Episode 10: Don't Map Twitter